In [1]:
import numpy as np 
import pickle 
import sys 
import time
from numpy.linalg import norm 
from matplotlib import pyplot as plt

In [3]:
import pandas as pd

In [5]:
mainpath = "C:\\Users\\leonr\\Downloads\\"

In [6]:
df = pd.read_csv(mainpath + "yelp.csv")

In [9]:
print('-Dataset examples-')
print(df.iloc[::500000, :])

-Dataset examples-
                        user_id    name  review_count yelping_since  useful  \
0        JJ-aSuM4pCFPdkfoZ34q0Q   Chris            10    2013-09-24       0   
500000   XgiawqkRRDp8XLCZ3GXQVw  Justin             4    2012-09-21       1   
1000000  c-agaB7J539HMXveaq8TWw  Angela             7    2011-04-08       1   

         funny  cool  fans elite  average_stars        ...          \
0            0     0     0  None           3.70        ...           
500000       0     0     0  None           4.50        ...           
1000000      0     0     0  None           4.43        ...           

         compliment_more  compliment_profile  compliment_cute  \
0                      0                   0                0   
500000                 0                   0                0   
1000000                0                   0                0   

         compliment_list  compliment_note  compliment_plain  compliment_cool  \
0                      0                0 

In [14]:
df1=df.drop(columns=['user_id', 'name',  'yelping_since', 'elite'])

In [15]:
print('-Dataset examples-')
print(df1.iloc[::50000, :])

-Dataset examples-
         review_count  useful  funny  cool  fans  average_stars  \
0                  10       0      0     0     0           3.70   
50000              13      12      1     2     0           5.00   
100000             16       1      1     0     0           3.44   
150000             16       2      0     0     0           3.82   
200000             46       3     16     0     3           3.79   
250000              1       2      0     0     0           1.00   
300000             21      23     21     7     1           3.68   
350000              2       0      0     0     0           1.00   
400000              3       0      0     0     0           1.00   
450000             43      14     11     5     5           3.35   
500000              4       1      0     0     0           4.50   
550000             16       4      2     0     0           3.65   
600000             31       1      0     0     0           4.50   
650000            321       7      4     3 

In [16]:
print(df1.shape)

(1326100, 17)


In [18]:
def get_closest(p, centers):
    '''
    Return the indices the nearest centroids of `p`.
    `centers` contains sets of centroids, where `centers[i]` is
    the i-th set of centroids.
    '''
    best = [0] * len(centers)
    closest = [np.inf] * len(centers)
    for i in range(len(centers)):
        for j in range(len(centers[0])):
            temp_dist = norm(p - centers[i][j])
            if temp_dist < closest[i]:
                closest[i] = temp_dist
                best[i] = j
    return best

In [57]:
def random_centers(rows, K):
  
    center = np.zeros(K)
    center_set = set()
    for i in range(0, K):
        center[i] = np.random.randint(rows)
        if center[i] in center_set:
            while center[i] in center_set:
                center[i] = np.random.randint(rows)
        else:
            center_set.add(center[i])
    return center

In [55]:
def minibatch_kmeans(df, k, minibatch_size, iters):
    # randomly pick k centroids
    centroids = df[np.array(random_centers(df.shape[0], k)).astype(int)]
    dic = dict()
    total_dis = 0
    for idx in range(1, iters + 1):
        batch = np.array(random_centers(df.shape[0], minibatch_size)).astype(int)
        for i in batch:
            min_dis = 10000000
            max_dis = 0

            # cache the center nearest to x
            for j in range(len(centroids)):
                temp_dis = np.linalg.norm(df[i, :] - centroids[j])
                if temp_dis < min_dis:
                    min_dis = temp_dis
                    dic[i] = j

            dis = np.linalg.norm(df[i, :] - centroids[dic[i]])
            total_dis += dis
            max_dis = max(max_dis, dis)
            min_dis = min(min_dis, dis)
        # update centers
        eta = 1 / idx
        for x in batch:
            centroids[dic[i]] = (1 - eta) * centroids[dic[i]] + eta * df[i, :]
            
    return centroids, dic

In [25]:
df2=df1.values

In [58]:
centroids, dic = minibatch_kmeans(df2, 200, 5000, 100)

In [46]:
print(centroids.shape)

(200, 17)


In [80]:
import random
def kmeans_initialize(df, K):
    Centers = []
    for i in range(K):
        Centers.append(df[np.random.randint(df.shape[0])])
    for k in range(1, K):
        cumprobs = 0
        D2 = np.array([min([np.inner(c-x,c-x) for c in Centers]) for x in df])
        probs = D2/np.sum(D2)
        cumprobs += probs
        r = random.random()
        for j,p in enumerate(cumprobs):
            if r < p:
                Centers[i] = df[j]
                break
        Centers.append(Centers[i])
    return Centers

In [76]:
def minibatch_kmeans_plus(df, k, minibatch_size, max_iter):
    # randomly pick k centroids
    centroids = kmeans_initialize(df,k)
    dic = dict()
    total_dis = 0
    for idx in range(1, iters + 1):
        batch = np.array(random_centers(df.shape[0], minibatch_size)).astype(int)
        for i in batch:
            min_dis = 10000000
            max_dis = 0

            # cache the center nearest to x
            for j in range(len(centroids)):
                temp_dis = np.linalg.norm(df[i, :] - centroids[j])
                if temp_dis < min_dis:
                    min_dis = temp_dis
                    dic[i] = j

            dis = np.linalg.norm(df[i, :] - centroids[  dic[i]])
            total_dis += dis
            max_dis = max(max_dis, dis)
            min_dis = min(min_dis, dis)
        # update centers
        eta = 1 / idx
        for x in batch:
            centroids[dic[i]] = (1 - eta) * centroids[dic[i]] + eta * df[i, :]
            
    return centroids, dic

In [ ]:
centroids, dic= minibatch_kmeans_plus(df2, 200, 5000, 100)